In [1]:
# this gets the most recent tiles from aws and runs them against our model for a prediction
import pandas as pd
import rasterio
import boto3
import numpy as np
import rasterio as rio
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject, Resampling
import tensorflow as tf
from tqdm import tqdm
from datetime import datetime, timedelta

In [2]:
mgrs = pd.read_csv('ca_mgrs.csv')['MGRS'].unique().tolist()

In [3]:
# get most recent prefix
def get_latest_prefix(mgrs):
    first = mgrs[0:2]
    second = mgrs[2:3]
    third = mgrs[3:5]
    bucket = 'sentinel-cogs'
    prefix = f'sentinel-s2-l2a-cogs/{first}/{second}/{third}/2022/9/'
    

    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
    all = response['Contents'] 
    #print(all)       
    for i in all:
        if i['Key'].endswith('TCI.tif'):
            if i['Size'] > 270000000:
                if i['LastModified'].date() > (datetime.now() - timedelta(days=15)).date():
                    latest_prefix = i['Key'].rsplit('/', 1)[0]
                else:
                    pass


    return latest_prefix


In [4]:
#downlaod B04, B08 and B12
def get_raster(latest_prefix, path):
    s3 = boto3.client('s3')
    bucket = 'sentinel-cogs'
    file_prefix = latest_prefix.split('/')[-1]
    B04 = path + 'process/' + f'{file_prefix}_B04.tif'
    B08 = path + 'process/' + f'{file_prefix}_B08.tif'
    B12 = path + 'B12/' +f'{file_prefix}_B12.tif'
    s3.download_file(bucket, latest_prefix + '/B04.tif', B04)
    s3.download_file(bucket, latest_prefix + '/B08.tif', B08)
    s3.download_file(bucket, latest_prefix + '/B12.tif', B12)
    return B04, B08, B12

In [5]:
def create_ndvi(B04, B08):
    with rio.open(B04) as src:
        Red = src.read(out_shape=(src.count, 5490, 5490), resampling=Resampling.bilinear).astype('float32')

    with rio.open(B08) as src:
        NIR = src.read(out_shape=(src.count, 5490, 5490), resampling=Resampling.bilinear).astype('float32')

    	
    np.seterr(divide='ignore', invalid='ignore')
    NDVI = (NIR-Red)/(NIR+Red)
    
    # convert nan to 0 (this necessary due to the divide by 0 issue)
    NDVI = np.nan_to_num(NDVI, nan=0, posinf=0, neginf=0)
    return np.squeeze(NDVI)
    

In [6]:
#function that takes NDVI, splits it and then return it into its orginal shape
def predict_fire(NDVI, model):
    #splits
    M = NDVI.shape[0]//10
    N = NDVI.shape[1]//10
    NDVI_split = [NDVI[x:x+M,y:y+N] for x in range(0,NDVI.shape[0],M) for y in range(0,NDVI.shape[1],N)]

    #do prediction here
    preds = []
    for ndvi in NDVI_split:
        new_prediction = model.predict(np.expand_dims(ndvi, axis=0))
        pred = np.squeeze(new_prediction, axis=0)
        preds.append(pred)
    #rejoins preds
    nd_arr = np.array(preds)
    nd_arr = nd_arr.reshape(10, 10, 549, 549)
    nd_arr = nd_arr.swapaxes(1,2)
    pred_B12 = nd_arr.reshape(5490, 5490)
    pred_B12[np.abs(pred_B12) < .01] = 0
    pred_B12 = pred_B12*1000

    return pred_B12

In [7]:
def create_ee_B12(pred_B12, real_B12_path, perd_B12_save_path):
    with rio.open(real_B12_path) as src:
        if src.crs == 'EPSG:32611':
            espg_path = '32611'
        else:
            espg_path = '32610'
        meta = src.meta
        #print(meta)
        meta['width'] = 5490
        meta['height'] = 5490
    

    pred_b12_file_name = perd_B12_save_path + espg_path + '/'+ real_B12_path.split('/')[-1]
# figure this out lol
#open B12 get meta from it (see above, and save new raster)
    with rio.open(pred_b12_file_name, 'w', **meta) as dst:
        dst.write(pred_B12, 1)
        dst.close


In [9]:
model_name = "model_16"
model = tf.keras.models.load_model(model_name)

In [10]:
for tile in tqdm(mgrs):
    try:
        latest_prefix= get_latest_prefix(tile)
        B04, B08, B12 = get_raster(latest_prefix, r'C:/temp/')
        NDVI = create_ndvi(B04, B08)
        pred_B12 = predict_fire(NDVI, model)
        create_ee_B12(pred_B12, B12, r'C:/temp/pred_B12/')
    except:
        print(tile)

 57%|█████▋    | 27/47 [10:45<05:49, 17.49s/it]

11SNU


100%|██████████| 47/47 [19:31<00:00, 24.92s/it]
